# Understanding the GUI (and related) part of the DC_Calibration

Refs: 
* https://github.com/ShivaramKPA/DC_Calibration/tree/master/src
* https://github.com/ShivaramKPA/VariousTests/tree/master/src/swing/consoles (The one that starts from [DC_App.java](https://github.com/ShivaramKPA/VariousTests/blob/master/src/swing/consoles/DC_App.java))

## Table of Contents
* 

### Program Starts by Opening the GUI in EDT
The program starts from the main method in Class '[DC_App.java](https://github.com/ShivaramKPA/VariousTests/blob/master/src/swing/consoles/DC_App.java)' (name may change later) and the method has only one line to create an object of **DC_GUI** class as follows:

```java
DC_GUI mk = new DC_GUI();
```
And the [**DC_GUI** class](#DC_GUI_Class) does nothing but defines, initializes and makes visible the GUI with all the components (after adding all the action listeners). No action really happens until we start clicking on or interacting with various components of the GUI.

Please note that this GUI opens in the **Event Dispatch Thread (EDT)** just as any SWING windows would do and this thread is different from the **main** thread which will exit early because in Java, it's possible for a program/process to survive even after the **main** thread exits if there is at least one non-Daemon thread still alive & running, which in this case is **EDT** at least.

**Note:** The **EDT** starts by default in either of the following two ways:
* When we invoke **.setVisible(true)** method on a frame object (which is what has been done in our case here)
* Or when we invoke the **SwingUtilities.invokeLater(..)** method passing a Runnable() implemented object as follows:
```java
    public static void main(String[] args) {
        SwingUtilities.invokeLater(new Runnable() {

            @Override
            public void run() {
                new CreateAndInitializeGUI();
            }

        });
}
```

<a id='DC_GUI_Class'></a>
### DC_GUI class - Defining, Initializing and Displaying the GUI
Here is what this class does:
* Defines the GUI windows and accessories/componets (panels, labels, buttons, textfields etc)
    * Prepare the main JFrame object for the main window
    * Create File Chooser and add it to the main window
    * Display the main window with its components
* Launch two threads with '**this**' object passed as the argument to each. These two threads run in parallel and redirect the standard-output and standard-error to our own console/text area respectively, with the help of objects of types **PipedInputStream and PipedOutputStream**. These two threads are start()ed immediately after the main GUI is made visible (displayed), which means the 'run()' method (the implemented method of the Runnable interface) of this object will be executed by both of these threads.
    * Please note that the passing of 'this' means, the reference to the same GUI object of type DC_GUI (not two different objects of the same type) is passed to these threads, thus the two threads will be interacting with the same object concurrently. For example, in our case, the 'this' reference is equivalent to 'mk'. 
        * So, it is important to remember that the window we open is a JFrame object member of bigger object 'mk' and the reference to that same object is what is passed to the two piped-stream threads.
* And adds various action-listeners to the window itself as well as to various components.

Here we define the class **DC_GUI** extending **WindowAdapter** and implementing the following three interfaces - **WindowListener, ActionListener, Runnable** as follows:
```java
public class DC_GUI extends WindowAdapter implements WindowListener, ActionListener, Runnable {
 ...
}
```

#### So, why do we extend WindowAdapter or Implement  WindowListener, ActionListener, Runnable?
* <font color='red'>First off, why extend **WindowAdapter** abstract class?</font>
    - The **windowClosed(..), windowClosing(..)** etc are some of the (synchronized) methods of the WindowListener interface. We don't have to implement all the methods of this interface here because this class also extends WindowAdapter which is an abstract adapter class for receiving window events. The methods in this abstract class are empty and the class exists as convenience for creating listener objects. 
        - If you implement the WindowListener interface, you have to define all of the    methods in it. This abstract class defines null methods for them all, so you can only have to define methods for events you care about.
        - see https://docs.oracle.com/javase/7/docs/api/java/awt/event/WindowListener.html and https://docs.oracle.com/javase/7/docs/api/java/awt/event/WindowAdapter.html The latter webpage says the following about WindowAdapter abstract class:
            - Create a listener object using the extended class and then register it with a Window (frame in our case) using the window's addWindowListener method (for example see the line 'frame.addWindowListener(this)' above. When the window's status changes by virtue of being opened, closed, activated or deactivated, iconified or deiconified, the relevant method in the listener object ('this' object in this/our case) is invoked, and the WindowEvent is passed to it.
    - If we don't use the WindowAdapter and only depend on JFrame, we can still make it exit on closing of the window by invoking the following line **frame.setDefaultCloseOperation(JFrame.EXIT_ON_CLOSE);**, but we cannot make it do anything more when it is ready to close or when it is closed already. With the extension of the WindowAdapter abstract class, now we can add any functionality we want putting the corresponding code inside the methods **windowClosed(..), windowClosing(..)**. For example, inside the synchronzed **windowClosed(WindowEvent ev)** method of the current DC_GUI case, we put code for invoking the 'join()' methods on the two **redirection** threads 'reader' and 'reader2'and also for closing the pipedStream objects 'pin' and 'pin2'. Likewise, inside the synchronized method **windowClosing(WindowEvent ev)** we put only two lines as follows:
```java
        frame.setVisible(false); // default behaviour of JFrame
        frame.dispose();
```

* <font color='red'>Next, why implement the **WindowListener** interface?</font> Isn't it superfluous to implement it if we are also extending the WindowAdapter?
    * I think that indeed seems to be the case. I mean the implementation of WindowListener is superfluous indeed. I simply removed the WindowListener from the class declaration and executed the program and it still ran as before with no difference or side effects.
* <font color='red'>Next, why implement the **ActionListener** interface?</font> Is it also avoidable?
    * Because of the implementation of this interface, we have the method **public synchronized void actionPerformed(ActionEvent evt) ** implemented but it seems not much is done inside this method. So, may be we could live without implementing this interface. I need to think more about it.
* <font color='red'>Next, why implement the **Runnable** interface?</font> Is it also avoidable?  <a id='implementsRunnable'></a> 
    * The overriden **run()** method is associated with the Runnable and the method is executed when the **start()** method is invoked on the thread which was created by passing the **this** object. In our case we create two such threads 'reader' and 'reader2' which do the grunt work of redirecting the standard outputs and errors to our own text area.
        * And, inside the [run()](#runMethod) method we have two identical if-blocks - one for thread 'reader' and another for 'reader2' - that takes care of how to process and where to display the redirected standard outputs and errors. The 'reader' redirects standard-output with the help of PipedInputStream object 'pin' and 'reader2' redirects standard-errors using 'pin2'. Please note that these two threads run in the background because they are made 'daemon' threads in the beginning by invoking their methods 'setDaemon(true)' (which is done just before the start() method is called).

#### Data Members of the DC_GUI class

## Appendix

<a id='runMethod'></a>
### The run() method 
This is the method that must be defined/implemented if a given class implements the Runnable interface. Please note that our class [**DC_GUI** implements](#implementsRunnable) this interface in addition to others.

<font color="brown">
This method defines the **Tasks** to be done by two **background/daemon** threads 'reader' and 'reader2'. The tasks for each of the two threads are almost identical as follows:
* As soon as the thread is started, it goes into the while-loop with it's condition checking for which thread it is.
* During each iteration, it sits quiet (waits) for 100 milli seconds (1/10th of a second) doing nothing.
* As the 100 ms wait is over, it retrieves the bytestream from pin or pin2, which is then parsed by the local function 'readLine(pin or pin2)' and then dumped onto the textArea of our own.
* At the end of each iteration, it also checks whether the value of 'quit' has flipped to 'true' or not. If it did flip, the thread returns or exits, otherwise, it starts another iteration of the while-loop that start by waiting for 100 ms as stated above. 
</font>

Here, please understand the following things:
* What do we need synchronized in front of run()?
* Why do we use wait() method
* What is available() method for?
* I think the local function '[readLine(PipedInputStream in)](#readLineMethod)' is just for the purpose of parsing the bytes coming in through either 'pin' or 'pin2'.
* When does the 'quit' variable become true?
    * It's value is set to 'true' at the very beginning of the "public synchronized void **windowClosed(WindowEvent evt)**" method, which is one of the overriden methods of the **WindowAdapter** abstract class which is extended by our [**DC_GUI**](#extendsWindowAdapter) class. In fact, windowClosed(WindowEvent evt) is one of the methods comes from the WindowListener interface which is implemented by WindowAdapter itself.
```java
    public synchronized void run() {
        try {
            while (Thread.currentThread() == reader) {
                try {
                    this.wait(100);
                } catch (InterruptedException ie) {
                }
                if (pin.available() != 0) {
                    String input = this.readLine(pin);
                    textArea.append(input);
                }
                if (quit) {
                    return;
                }
            }

            while (Thread.currentThread() == reader2) {
                try {
                    this.wait(100);
                } catch (InterruptedException ie) {
                }
                if (pin2.available() != 0) {
                    String input = this.readLine(pin2);
                    textArea.append(input);
                }
                if (quit) {
                    return;
                }
            }
        } catch (Exception e) {
            textArea.append("\nConsole reports an Internal error.");
            textArea.append("The error is: " + e);
        }

        /*
		 * // just for testing (Throw a Nullpointer after 1 second) if (Thread.currentThread()==errorThrower) { try { this.wait(1000); }catch(InterruptedException
		 * ie){} throw new NullPointerException( "Application test: throwing an NullPointerException It should arrive at the console" ); }
         */
}
```

<a id='readLineMethod'></a>
### The local 'readLine(PipedInputStream in)' method
Here, please understand the following:
* available() and read(byteArray) methods of PipedInputStream
```java
    public synchronized String readLine(PipedInputStream in) throws IOException {
        String input = "";
        do {
            /**
             * kp: PipedInputStream inherits from InputStream and available() is
             * one of its methods.
             * https://docs.oracle.com/javase/7/docs/api/java/io/InputStream.
             * html available(): Returns an estimate of the number of bytes that
             * can be read (or skipped over) from this input stream without
             * blocking by the next invocation of a method for this input
             * stream.
             *
             * read(byte[] b): Reads some number of bytes from the input stream
             * and stores them into the buffer array b.
             */
            int available = in.available();
            if (available == 0) {
                break;
            }
            byte b[] = new byte[available]; // kp: creating a 'byte' array of
            // size 'available'
            in.read(b);
            input = input + new String(b, 0, b.length);
        } while (!input.endsWith("\n") && !input.endsWith("\r\n") && !quit);
        return input;
    }
```


### The windowClosed(WindowEvent evt) method
```java
    //
    // kp: 3/3/18:
    //The windowClosed(..), windowClosing(..) etc are some of the methods of
    // the WindowListener interface. We don't have to implement all the methods of this
    // interface here because this class also extends WindowAdapter which is an abstract
    //  adapter class for receiving window events. The methods in this abstract class  
    //   are empty and the class exists as convenience for creating listener objects. 
    //  (If you implement the WindowListener interface, you have to define all of the 
    //     methods in it. This abstract class defines null methods for them all, so you can 
    //     only have to define methods for events you care about.) 
    // see https://docs.oracle.com/javase/7/docs/api/java/awt/event/WindowListener.html 
    // and https://docs.oracle.com/javase/7/docs/api/java/awt/event/WindowAdapter.html 
    //  The latter webpage says the following about WindowAdapter abstract class:
    //      Create a listener object using the extended class and then register it with a 
    //      Window (frame in our case) using the window's addWindowListener method (for 
    //      example see the line 'frame.addWindowListener(this)' above. When the window's status 
    //      changes by virtue of being opened, closed, activated or deactivated, iconified 
    ///     or deiconified, the relevant method in the listener object ('this' object in 
    //      this/our case) is invoked, and the WindowEvent is passed to it.
    //
    /*
     https://www.java-forums.org/new-java/31363-windowclosed-vs-windowclosing.html
        windowClosed(): After window has closed.
        windowClosing(): The process of being closed.    
    */
    public synchronized void windowClosed(WindowEvent evt) {
        quit = true;
        // https://docs.oracle.com/javase/7/docs/api/java/lang/Object.html
        // notify() :   Wakes up a single thread that is waiting on this object's monitor.
        // notifyAll() : Wakes up all threads that are waiting on this object's monitor.
        this.notifyAll(); // stop all threads (kp: notify All threads?)
/*        
        // kp:
        // https://docs.oracle.com/javase/tutorial/essential/concurrency/join.html

        The join method allows one thread to wait for the completion of another. If t is 
        a Thread object whose thread is currently executing,
        
        t.join();

        causes the current thread to pause execution until t's thread terminates. 
        Overloads of join allow the programmer to specify a waiting period. However,
        as with sleep, join is dependent on the OS for timing, so you should not assume 
        that join will wait exactly as long as you specify.

        Like sleep, join responds to an interrupt by exiting with an InterruptedException.
        
        https://docs.oracle.com/javase/8/docs/api/java/lang/Thread.html
        join() : Waits for this thread to die.
        join(long millis) : Waits at most millis milliseconds for this thread to die.
        join(long millis, int nanos)  : Waits at most millis milliseconds plus nanos 
        nanoseconds for this thread to die.
         */       
        
/*
* https://www.youtube.com/watch?v=vLjucKGR654 JOIN() METHOD : JAVA MULTITHREADING TUTORIALS
*   3/8/18:
*  Join() method ensures termination of thread which calls its join method 
*   before any other thread such as (main thread) terminates.
*   This method can be used to decide sequence of thread's execution.
*        
*  Whenever a thread calls the join() method from another thread, the 'hosting
*    thread' (from which the call is made) has to wait until that thread (whose 
*    join method is called) completes it's execution.
*
         */       
        try {            
            //kp: Following join method asks the current thread (i.e. the EDT thread - in 
            //    which our Swing windows/frames run) to wait (i.e. not move ahead or exit 
            //    early) until the reader thread terminates/exits.
            reader.join(1000);
            pin.close();
        } catch (Exception e) {
        }
        try {
            reader2.join(1000);
            pin2.close();
        } catch (Exception e) {
        }
        System.exit(0);
    }

    public synchronized void windowClosing(WindowEvent evt) {
        frame.setVisible(false); // default behaviour of JFrame
        frame.dispose();
    }
```